# **Çağdaş Güven 2738938**

# **ME 536**

### Find and count characters and their elements in strings
Check if you can see the hidden ***SVD flavor*** somewhere in the requirements!

In [ ]:
# only importing from following libraries are allowed. You can add more imports from them, but no new libraries here
from skimage import io
from skimage.filters import threshold_otsu as otsu
import numpy as np
from scipy.linalg import orth
from scipy.spatial import distance_matrix
from scipy.ndimage import gaussian_filter
from scipy.sparse.csgraph import laplacian
from scipy.linalg import eigh
from numpy.linalg import matrix_rank as rank
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import normalize, StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import TruncatedSVD
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
from sklearn.cluster import SpectralClustering
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances, rbf_kernel
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans # just for demo purposes, you can import more if needed...

# also import the matrix printing function
!rm bug_numpy_utils.py 2> dump.me ## change these to ! when uploading the file
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/bug_numpy_utils.py  ## change these to ! when uploading the file
from bug_numpy_utils import CData as CMe
from bug_numpy_utils import GenerateDataforImage as GenImMat
from bug_numpy_utils import text2mat

# Intro to Basics: Assignment has not started yet!
This is the warm up

## Generate and plot reference text
Generate data matrix from a string.

Columns of this matrix are data points, which when plotted is read as the given string.  

Using pyplot display the data points to make sure that they are readable.  

The problem is given in 2D below, play with the ```NoiseLevel``` and observe how data points mere into each other.

In [ ]:
# this is a support function to see the result of clustering better

def ColorizeChars(M, Mnum = [], Title='some string', figSize = (9, 3),  aspectR = 2):
    Indicies = np.hstack((np.array([0]), np.cumsum(Mnum)))
    fig, ax = plt.subplots(figsize= figSize) # Increased figure size
    # absence of Mnum is that we do not want to colorize the plot
    if Mnum is None or Mnum is [] or len(Mnum) == 0:
        ax.plot(M[0,:],M[1,:], '*')
    else:
        for i, uLim in enumerate(Indicies):
            if i < len(Indicies)-1:
                X = M[0, uLim:Indicies[i+1]]
                Y = M[1, uLim:Indicies[i+1]]
                #plt.plot(X,Y, '*')
                ax.plot(X,Y, '*')
    ax.set_aspect(aspectR) # Set the aspect ratio to aspectR:1
    ax.set_title(Title)
    plt.show()

In [ ]:
S1 = 'hello clustering'
T1, T1num = text2mat(S1)

# just that we get to understand ```text2mat``` function let's print the output of the function
print(f'Shape of T1 = {T1.shape}, where letters of "{S1}" has {T1num} data points in each corresponding letter')

NoiseLevel = 0.0

T1 += NoiseLevel * np.random.randn(*T1.shape)


ColorizeChars(T1) # just plot the data as a single chunk
ColorizeChars(T1, T1num, Title='clusters marked', figSize=(7,2), aspectR=4) # call this function anyway you like

### WARNING: Testing conditions is not Vanilla
Note that data matrix that will be sent might slightly be manipulated after it is generated with ```text2mat```, way beyond adding noise.  
Check out the following to give you an idea.

Note that when data matrix is shuffled, color printing makes no sense, because the columns are no more sorted, hence values returned by text2mat does not make sense.  


In [ ]:
# CELL 1
Stest = 'ahanda boyle'
Ttest, Tnum = text2mat(Stest)
noiseLevel = 0.9

#'''
# original text
V1 = Ttest[0:2, :]
ColorizeChars(V1, Title='Original text')
#'''

#'''
# contaminated ... play with noise levels
V2 = V1 + noiseLevel * np.random.randn(*V1.shape)
ColorizeChars(V2, Tnum, Title=f'Noisy level = {noiseLevel}')
#'''

#'''
# extended or shrung along X-Y axis
V3 = np.copy(V1)
Xscale = 0.4
Yscale = 1.2
V3[0,:] *= Xscale
V3[1,:] *= Yscale
ColorizeChars(V3, Tnum, Title=f'Scaled along X and Y by =[{Xscale}, {Yscale} ]')
#'''

#'''
V4 = np.copy(V3)
V4 = V4[:, np.random.permutation(V4.shape[1])]
ColorizeChars(V4, Tnum, Title='columns are shuffled, this is where fun begins')
#'''

#'''
V5 = orth(np.random.rand(2,2)) @ V2
ColorizeChars(V5, Tnum, Title='rotated randomly and noisy,  more fun :) ', aspectR=1)
#'''


#'''
V6 = V5[:, np.random.permutation(V5.shape[1])]
ColorizeChars(V6, Tnum, Title='shuffled and rotated and contaminated, almost the peak of fun', figSize=(7,7))
#'''

# Assignment starts here
Read the following cells carefully and respond by filling in the code and text cells. Your explanations brief yet clear.

This assignment will hopefully make you better in clustering simple cases :)  
For harder cases, we will talk about artificial neural networks...

## Let's start with a show case
Note that ```text2mat``` function returns a data matrix and a list.  
Also note that the data matrix is sorted, so that the first groups of points belong to the first letter, second group to the second letter and so on.  
The list contains the number of points in each groups that correspond to the letters in the text that is sent to the function.

So your objective indeed is to recover clusters, sort them, so that when we print, it plot them using what is returned from your function, it plots properly.  

However, note that your sorting algorithm is not necessarily expected to find the order of clusters (i.e. letters), it is possible that you read the string from the end to beginning. By running the following you will see that the text might even be mirrorred. Under any circumstance, you should be able to find the alignment of the text, it does not matter wheter it is backwards or mirrorred, cluster it and return the sorted matrix along with the number of elements in each cluster, similar to what ```text2mat``` does. When we plot it using ```ColorizeChars``` it should look meaningful.

In other words, your element count list should either be similar to what text2sum returns or to the inverse of the list.  

Run the following cell for different noise levels and observe the changes.


## Clustering time:
Using any approach you like sort points in the given data matrix.
You can use hence import other sub-libraries in already imported libraries above.  
No new libraries...

Objective is to see if you can find letters indiviually.  
In other words, after we shuffle everything, objective is to check if you can

### Implemet the sorting function: i.e. ```SortPoints()```
If you would like to seperate this function into smaller other functions, write them in the support functions cell

In [ ]:
# support functions goes in here so that I run them before SortPoints()
# write as many as needed
#

def find_optimal_k_dendrogram(M, max_k=25, threshold=150, debug=False):
    """
    Determine the optimal number of clusters (K) using Dendrogram analysis with a fixed distance threshold.

    Args:
        M (ndarray): Input data matrix (D x N).
        max_k (int): Maximum number of clusters to consider for truncating dendrogram.
        threshold (float): Distance threshold to determine clusters.
        debug (bool): If True, displays the dendrogram for visualization.

    Returns:
        optimal_k (int): Optimal number of clusters determined by dendrogram analysis.
    """
    # Transpose M if it's in D x N format
    if M.shape[0] < M.shape[1]:
        M = M.T

    # Compute the linkage matrix using Ward's method
    Z = linkage(M, method='ward')

    # Plot dendrogram if debugging is enabled
    if debug:
        plt.figure(figsize=(10, 8))
        dendrogram(Z, truncate_mode='lastp', p=max_k, show_leaf_counts=True, color_threshold=threshold)
        plt.axhline(y=threshold, color='r', linestyle='--', label=f"Threshold = {threshold}")
        plt.title("Dendrogram for Optimal K")
        plt.xlabel("Cluster Size")
        plt.ylabel("Distance")
        plt.legend()
        plt.grid()
        plt.show()

    # Determine the number of clusters based on the distance threshold
    optimal_k = sum(Z[:, 2] > threshold) + 1  # Count clusters above threshold

    if debug:
        print(f"Optimal K determined by dendrogram analysis with threshold {threshold}: {optimal_k}")

    return optimal_k



In [ ]:
# def SortPoints(M, K = -1):
#     '''
#     Columns of M are the data points
#     K is the number of clusters, if K = -1, this function is to find the number of clusters!

#     returns Ms, MsNum
#     Ms: the column sorted version of M, similar to what is given by text2mat before shuffle
#     MsNum: element count list --> the number of points in each cluster, similar to what text2mat returns
#     Note that len(MsNum) = number of letters in the original string
#     '''
#     # fill in the function so that...
#     Ms = None
#     MsNum = []


#     Ms, MsNum

In [ ]:


def SortPoints(M, K=-1):
    """
    SortPoints with hierarchical clustering and sorting without preprocessing.
    Args:
        M: Input data matrix (D x N).
        K: Number of clusters (-1 for automatic estimation).
        max_k: Maximum number of clusters to consider for elbow method.
        debug: Enable debug outputs.
    Returns:
        R: Sorted matrix of points.
        Rsum: List of points per cluster.
    """

    max_k = 50
    debug= False
    # Step 1: Estimate K using the elbow method if not provided
    if K == -1:
        K = find_optimal_k_dendrogram(M, max_k=max_k, debug=debug)

    #print(f"Estimated number of clusters (K): {K}")

    # Step 2: Perform Hierarchical Clustering
    # Compute the pairwise distance matrix
    pairwise_distances = pdist(M.T)
    # Perform hierarchical clustering
    Z = linkage(pairwise_distances, method='ward')
    # Assign cluster labels
    labels = fcluster(Z, K, criterion='maxclust')

    if debug:
        print(f"Cluster labels: {labels}")

    # Step 3: Sort clusters by centroid X-coordinates
    centroids = np.array([M[:, labels == i].mean(axis=1) for i in range(1, K + 1)])
    sorted_indices = np.argsort(centroids[:, 0])

    # Step 4: Group points by sorted clusters
    R = []
    Rsum = []
    for idx in sorted_indices:
        cluster_points = M[:, labels == (idx + 1)]  # Labels are 1-indexed
        R.append(cluster_points)
        Rsum.append(cluster_points.shape[1])

    R = np.hstack(R)  # Combine clusters into one matrix
    return R, Rsum



### Explanation of `SortPoints`

The `SortPoints` function clusters and sorts data points using hierarchical clustering based on the following steps:

1. **Cluster Number Estimation**:
   - If `K` (number of clusters) is not provided, the function estimates `K` using `find_optimal_k_dendrogram`.

2. **Hierarchical Clustering**:
   - Computes pairwise distances using the data matrix $ M $ (shape $ D \times N $).
   - Performs hierarchical clustering using Ward's method to minimize within-cluster variance.
   - Assigns cluster labels based on the computed linkage matrix and the determined number of clusters $ K $.

3. **Cluster Sorting**:
   - Calculates cluster centroids and sorts clusters by their $ X $-coordinates (first dimension of $ M $).
   - Groups points in sorted order and outputs the sorted matrix $ R $ and cluster sizes $ R_{\text{sum}} $.

---

### Explanation of `find_optimal_k_dendrogram`

The `find_optimal_k_dendrogram` function determines the optimal number of clusters ($ K $) using dendrogram analysis:

1. **Linkage Matrix**:
   - Computes a hierarchical clustering linkage matrix $ Z $ using Ward's method.

2. **Dendrogram Visualization** (optional):
   - Optionally visualizes the dendrogram with clusters highlighted by a distance threshold.

3. **Cluster Number Determination**:
   - Counts the number of clusters above the given distance threshold.
   - Returns this count as the optimal $ K $.

These two functions work together to cluster data and sort points based on hierarchical clustering, ensuring the clusters are sorted spatially.

### Let's test the sorting function

If you have implemented ```SortPints``` properly, following should work.
I will only call ```SortPints``` to test your work.


In [ ]:
# let's generate test data
St = 'Test Data'
Tt, Ttsum = text2mat(St)
#in my case I will generate variations of Tt as I did above but for simplicty here I will stick with Tt

#I will run one of the following
R, Rsum = SortPoints(Tt) # case when k- number of clusters is given -
# also check for fun what happens when k is given to be something different then the correct value

R, Rsum = SortPoints(Tt, len(Ttsum)) # case when it is not given

# finally see the result
ColorizeChars(R, Rsum)
